In [29]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import f
from scipy.spatial import distance
from scipy.stats import multivariate_normal

In [30]:
df = pd.read_excel('/Users/tema.fazly/Desktop/МИФИ/7 семестр/Домашова/2 Лабораторная /Лабораторня_2_Фазлы.xlsx',
                   sheet_name='Форматирование', usecols=['Наименование', 'k-means', 'X1', "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", 'training'],
                   index_col='Наименование')
df = df.drop(index=df.index[-1])
X_train = df[df['training'].notna()]
X_train

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training
Наименование,,,,,,,,,,,
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0
Брянская область,2,0.431122,-1.054390,-0.361607,-0.190445,-0.769025,-0.417424,-0.393798,-0.029124,0.034404,2.0
Волгоградская область,2,-0.270446,-0.497239,-0.239931,-0.221703,-1.133328,-0.527656,-0.030817,-0.545753,0.397112,2.0
г. Москва,4,2.136728,2.171223,-0.485636,-1.418893,-0.880231,0.552394,3.684966,-1.535959,0.181754,4.0
Ивановская область,2,-0.480496,-0.233325,-0.419465,-0.037756,-0.834095,-0.122032,0.150674,-0.136755,-0.827027,2.0
Калужская область,6,1.153694,-0.927321,-0.414311,-0.611723,-0.192756,-0.384876,-0.517976,-0.911699,-0.135615,6.0
Камчатский край,5,-1.572756,0.998273,-0.049462,0.691446,0.437701,2.454668,-0.880958,0.143086,-0.407646,5.0
Карачаево-Черкесская Республика,7,1.027664,-0.624309,-0.417856,-0.931149,0.127369,-1.321268,-0.107234,2.123498,-1.881147,7.0
Краснодарский край,6,0.540348,-0.370169,-0.317393,-1.053546,0.016381,0.696966,-0.603946,-0.674910,-0.044938,6.0


In [31]:
def compute_matrix(data, matrix_type='cov'):

    filtered_data = data[data['training'].notna()]
    
    features = filtered_data.loc[:, ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']]
    
    if matrix_type == 'cov':
        matrix = np.cov(features, rowvar=False)
    elif matrix_type == 'corr':
        matrix = np.corrcoef(features, rowvar=False)

    return pd.DataFrame(matrix, columns=features.columns, index=features.columns)

cov = compute_matrix(X_train)
cov

,X1,X2,X3,X4,X5,X6,X7,X8,X9
X1,0.808902,-0.238623,-0.146274,-0.511005,-0.380215,-0.491726,0.197206,0.013126,-0.228801
X2,-0.238623,1.150351,0.053935,0.022560,0.225839,0.815993,0.756199,-0.409249,0.075999
X3,-0.146274,0.053935,0.102149,0.223078,0.088309,0.111052,-0.113973,0.027507,0.144841
X4,-0.511005,0.022560,0.223078,0.681526,0.310023,0.298669,-0.400897,0.116221,0.427157
X5,-0.380215,0.225839,0.088309,0.310023,0.613225,0.418266,-0.111670,-0.054539,0.372280
X6,-0.491726,0.815993,0.111052,0.298669,0.418266,1.119835,0.177639,-0.433103,0.336616
X7,0.197206,0.756199,-0.113973,-0.400897,-0.111670,0.177639,1.222880,-0.305409,-0.215046
X8,0.013126,-0.409249,0.027507,0.116221,-0.054539,-0.433103,-0.305409,0.659509,-0.191843
X9,-0.228801,0.075999,0.144841,0.427157,0.372280,0.336616,-0.215046,-0.191843,0.787360


In [32]:

def discrim(data, factors, training, method):

    X = data[data[training].notna()]
    
    X_train = X[factors]
    Y_train = X[method]
    x_test = data[factors]
    
    
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, Y_train)
    
    y_pred = lda.predict(x_test)
    return y_pred

In [33]:
def scatter_matrix(samples):
    # является ли подклассом?
    # проверка если по каким то причинам наши значения признаков имеют тип данных series то их конвертирует в data frame
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    # вычитает из значений признаков средние значения признаков
    d = samples - samples.mean()
    # создаёт матрицу нулей размерностью 9 на 9
    res = np.zeros((d.shape[1], d.shape[1]))
    # приводит к виду int: 32, 24, ...
    for _, row in d.iterrows():  # проходимся циклом по каждой строке матрицы D датафрейм(там где от значений - срзнач)
        # берём строчку из таблицы D( которая сейчас имеет вид seria и мы приводим её к виду dataframe)
        col = row.to_frame()
        res += col @ col.T  # матрица из нулей 9х9 берём строчку из датафрейма и умножаем её на неё же только транспонированную и так проходим по всем строчкам
    return res


# передаём samples (значение признаков в обучающей выборке табличкой) и labels (номера классов) shape если 0 то число строк если 1 то число колонок
def classes_scatter_matrix(samples, labels):
    # zeros создаёт матрицу shape на shape то есть создаёт матрицу число признаков на число призноков  (9 на 9) заполненую нулями
    A = np.zeros((samples.shape[1], samples.shape[1]))
    for cls in labels.unique():  # переменная cls счётчик по классам принимает значения уникальных классов то есть у нас от 1 до 7
        # В матрицу А прибавляем соответствующие значения из матрицы полученые в результате работы skater matrix для текущего класса
        A += scatter_matrix(samples[labels == cls])
    return A

def wilks_lambda(samples, labels):  # samples  - на каждой итерации мы передаём в функцию wilks lmbd dataframe сначала с одной колонкой Х1 постепенно увеличивая число колонок пока не дойдём до конца   labels - колонка с номерами классов
    if isinstance(samples, pd.Series):
        samples = samples.to_frame()
    # определитель матрицы рассеивания
    dT = np.linalg.det(scatter_matrix(samples))
    # определитель классовой матрицы рассеивания
    dE = np.linalg.det(classes_scatter_matrix(samples, labels))
    return dE / dT

def f_p_value(lmbd, n_obj, n_sign, n_cls):
    num = (1-lmbd)*(n_obj - n_cls - n_sign)
    den = lmbd * (n_cls - 1)
    f_value = num / den
    p = f.sf(f_value, n_cls-1, n_obj-n_cls-n_sign)
    return f_value, p

def forward(samples, labels, f_in = 0.01):                                            # samples - значение признаков в обучаюзей выборке  labels - колонка с номерами классов  f_in точность( это F to inter в статистике)
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to enter", "P value"]             # создаётся список названий колонок таблицы
    n_cls = labels.unique().size                                                      #число уникальных классов нашей обучающей выборки
    n_obj = samples.shape[0]                                                          # число  объектов в обучающей выборке
                                                                                      # хранение пременных вне и в модели(е)
    out = {0: pd.DataFrame(columns=st_columns, index=samples.columns, dtype=float)}   # создаётся словарик ключу(ключ показывает какой у нас шаг метода) 0 ставится в соответствие дата фрейм(пустой) с колонками из переменной st_colums и индексами(строки таблицы) Х1,,,Х9
    into = {0: pd.DataFrame(columns=st_columns, dtype=float)}                         # создаётся словарик ключу(ключ показывает какой у нас шаг метода) 0 ставится в соответствие дата фрейм(пустой) с колонками из переменной st_colums
    step = 0                                                                          # шаг нашего метода

    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # расчёт характеристик элементов вне модели
        for el in out[step].index: # el переменная счётчик по списку индексов датафрейма внутри out (Х1 Х2,,, Х9)
            lmbda = wilks_lambda(samples[into[step].index.tolist() + [el]], labels)   # мы с датафрейма samples  берём значение из колонок в квадратных скобках список колонок который есть в inta для текущего шага + текущая колонку из счётчика el и эти значения помещаются в функцию вилкс лямбда
            partial_lmbd = lmbda / model_lmbd   #
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # расчёт характеристик элементов в моделе
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(partial_lmbd, n_obj, into[step].index.size-1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if out[step].index.size == 0 or out[step]["F to enter"].max() < f_in or step == 7:
            break
        # добавление нового элемента
        el_to_enter = out[step]["F to enter"].idxmax()
        into[step+1] = pd.concat([into[step], out[step].loc[[el_to_enter]]])
        out[step+1] = out[step].drop(index=el_to_enter)

        step += 1
    return into, out


into, out = forward(X_train.loc[:, 'X1':'X9'], X_train['training'], f_in = 2 )

forw_stepwise = into[len(into) - 1].index.tolist()
forw_stepwise


['X7', 'X4', 'X8', 'X2', 'X1', 'X9']

In [34]:
def backward(samples, labels, f_r=13.00):
    st_columns = ["Wilk's lmbd", "Partial lmbd", "F to remove", "P value"]
    n_cls = labels.unique().size
    n_obj = samples.shape[0]
    # хранение пременных вне и в модели(е)
    into = {0: pd.DataFrame(
        columns=st_columns, index=samples.columns, dtype=float)}
    out = {0: pd.DataFrame(columns=st_columns, dtype=float)}
    step = 0

    while True:
        model_lmbd = wilks_lambda(samples[into[step].index], labels)
        # расчёт характеристик элементов вне модели
        for el in out[step].index:
            lmbda = wilks_lambda(
                samples[into[step].index.tolist() + [el]], labels)
            partial_lmbd = lmbda / model_lmbd
            f_lmbd, p_value = f_p_value(
                partial_lmbd, n_obj, into[step].index.size, n_cls)
            out[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value
        # расчёт характеристик элементов в моделе
        for el in into[step].index:
            lmbda = wilks_lambda(samples[into[step].index.drop(el)], labels)
            partial_lmbd = model_lmbd / lmbda
            f_lmbd, p_value = f_p_value(
                partial_lmbd, n_obj, into[step].index.size-1, n_cls)
            into[step].loc[el] = lmbda, partial_lmbd, f_lmbd, p_value

        if into[step].index.size == 0 or into[step]["F to remove"].min() > f_r or step == 2:
            break
        # удаление элемента
        el_to_remove = into[step]["F to remove"].idxmin()
        out[step+1] = pd.concat([out[step], into[step].loc[[el_to_remove]]])
        into[step+1] = into[step].drop(index=el_to_remove)

        step += 1
    return into, out

into, out = backward(X_train.loc[:, 'X1':'X9'], X_train['training'], f_r=15)

forw_backward = into[len(into) - 1].index.tolist()
forw_backward = forw_stepwise

In [35]:
def accuracy(data, training, predict):

    curacy_dis_analys = (data[training] == data[predict]).astype(int)
    len_train = len(df[df[training].notna()])
    accuracy = curacy_dis_analys.sum() / len_train
    return accuracy


In [36]:
def predict(data, method):
    clusters = {}
    for cluster, group in data.groupby(method):
        clusters[cluster] = group.index.tolist()

    for cluster, group in clusters.items():
        print(f'Кластер {cluster}: {group}')

In [37]:
def scatter_matrix(data):
    "Вычисляет матрицу рассеивания"
    
    mean_vector = data.mean(axis=0).values
    centered_data = data - mean_vector
    return np.dot(centered_data.T, centered_data)

pd.DataFrame(scatter_matrix(X_train.loc[:, "X1":"X9"]))

,0,1,2,3,4,5,6,7,8
0,21.840350,-6.442824,-3.949397,-13.797142,-10.265806,-13.276602,5.324552,0.354405,-6.177628
1,-6.442824,31.059469,1.456256,0.609115,6.097655,22.031807,20.417368,-11.049729,2.051964
2,-3.949397,1.456256,2.758017,6.023098,2.384355,2.998410,-3.077270,0.742689,3.910719
3,-13.797142,0.609115,6.023098,18.401193,8.370623,8.064071,-10.824220,3.137980,11.533245
4,-10.265806,6.097655,2.384355,8.370623,16.557068,11.293178,-3.015077,-1.472558,10.051556
5,-13.276602,22.031807,2.998410,8.064071,11.293178,30.235551,4.796259,-11.693789,9.088632
6,5.324552,20.417368,-3.077270,-10.824220,-3.015077,4.796259,33.017768,-8.246047,-5.806251
7,0.354405,-11.049729,0.742689,3.137980,-1.472558,-11.693789,-8.246047,17.806747,-5.179766
8,-6.177628,2.051964,3.910719,11.533245,10.051556,9.088632,-5.806251,-5.179766,21.258708


In [38]:
def classes_scatter_matrix(data, labels):
    "Вычисляет классовую матрицу рассеивания"
    
    class_means = data.groupby(labels).mean()
    overall_mean = data.mean()
    
    scatter_matrix = np.zeros((data.shape[1], data.shape[1]))
    
    for label, group in data.groupby(labels):
        n = len(group)
        mean_diff = class_means.loc[label] - overall_mean
        scatter_matrix += n * np.outer(mean_diff, mean_diff)
    
    return scatter_matrix

In [39]:
df["Dis_analys"] = discrim(df, ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9'], 'training', 'k-means')
predict(df, 'Dis_analys')
print(f'Точность Дискриминантного анализа = {accuracy(df, "training", "Dis_analys").round(3)}')
df

Кластер 1: ['Алтайский край                          ', 'Костромская область                     ', 'Курская область                         ', 'Мурманская область                      ', 'Новгородская область                    ', 'Оренбургская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Рязанская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тверская область                        ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Калининградская область', 'Кировская область                       ', 'Курганская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys
Наименование,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5


In [40]:
mean_values = df.loc[:, 'X1':'X9'].groupby(df['training']).mean().reset_index().loc[:, 'X1':'X9']
mean_values

,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,-0.084201,-0.091594,-0.203627,-0.065107,0.560224,-0.005142,0.080625,-0.215684,0.801380
1,-0.182225,-0.416250,-0.299947,-0.163088,-0.589765,-0.382497,0.102913,-0.038349,-0.119423
2,-0.497300,0.291244,-0.302030,-0.467726,0.085368,0.373283,1.013550,-0.366368,-0.570109
3,0.973051,3.021612,-0.474449,-1.434522,-0.465548,1.350706,3.470043,-1.589775,-0.503990
4,-0.952688,0.869248,0.315369,1.102731,0.589081,1.369091,-0.523708,-0.011903,0.648740
5,0.931041,-0.608018,-0.391132,-0.951607,-0.311724,0.187218,-0.852301,-1.004979,-0.124281
6,1.561191,-1.088601,-0.454279,-1.310194,-0.968260,-2.015285,-0.078578,1.843658,-1.920818


In [41]:
mean_values_Dis_analys = df[df['training'].notna()]
mean_values_Dis_analys = mean_values_Dis_analys.groupby('Dis_analys').mean().reset_index().loc[:, 'X1':'X9']

mean_values_Dis_analys

,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,0.018023,-0.063899,-0.189689,-0.025391,0.474734,-0.141386,-0.013305,-0.043475,0.869387
1,-0.244539,-0.449995,-0.373435,-0.225508,-0.608491,-0.425037,0.141122,-0.079352,-0.250851
2,-0.478395,0.123447,-0.212186,-0.357512,0.072909,0.452514,0.869472,-0.481174,-0.362308
3,0.973051,3.021612,-0.474449,-1.434522,-0.465548,1.350706,3.470043,-1.589775,-0.503990
4,-0.952688,0.869248,0.315369,1.102731,0.589081,1.369091,-0.523708,-0.011903,0.648740
5,0.931041,-0.608018,-0.391132,-0.951607,-0.311724,0.187218,-0.852301,-1.004979,-0.124281
6,1.561191,-1.088601,-0.454279,-1.310194,-0.968260,-2.015285,-0.078578,1.843658,-1.920818


In [42]:
def mahalanobis_distances(centroids, subjects) -> pd.DataFrame:

    # Получаем матрицу ковариаций
    cov_matrix = np.cov(subjects.T)
    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # Создаем DataFrame для хранения результатов
    distances = pd.DataFrame(index=subjects.index, columns=centroids.index)

    # Вычисление расстояний Махаланобиса
    for cluster_idx in centroids.index:
        centroid = centroids.loc[cluster_idx].values
        for subject_idx in subjects.index:
            subject = subjects.loc[subject_idx].values
            distances.at[subject_idx, cluster_idx] = (distance.mahalanobis(subject, centroid, inv_cov_matrix) ** 2)

    return distances

In [43]:
def mahalanobis_classes(distances_squared): #выводит субъекты класстеров по Махаланобису
    classes = distances_squared.idxmin(axis=1)
    return classes

In [44]:
def posterior_classes(probabilities: pd.DataFrame):
    # Находим индексы (классы) максимальных значений вероятностей
    classes = probabilities.idxmax(axis=1)
    return classes

In [45]:
def posterior_probabilities(centroids: pd.DataFrame, subjects: pd.DataFrame) -> pd.DataFrame:
    # Получаем число классов и число признаков
    num_clusters = centroids.shape[0]
    num_features = centroids.shape[1]

    # Предположим, ковариационная матрица одинаковая для всех кластеров
    # Чтобы упростить пример, используем ковариацию от всех субъектов
    cov_matrix = np.cov(subjects.T)
    
    # Проведение проверки на сингулярность ковариационной матрицы
    if np.linalg.cond(cov_matrix) < 1 / np.finfo(cov_matrix.dtype).eps:
        # Инициализация матрицы вероятностей
        probabilities = pd.DataFrame(index=subjects.index, columns=centroids.index)

        # Вычисление апостериорных вероятностей для каждого субъекта и кластера
        for cluster_idx in centroids.index:
            centroid = centroids.loc[cluster_idx].values
            # Вычисляем вероятности для всех субъектов по текущему центроиду
            probabilities[cluster_idx] = multivariate_normal.pdf(subjects, mean=centroid, cov=cov_matrix)

        # Нормализуем вероятности, чтобы сумма по каждому субъекту равнялась 1
        normalized_probabilities = probabilities.div(probabilities.sum(axis=1), axis=0)

        return normalized_probabilities

In [46]:
mahalanobis = mahalanobis_distances(centroids=mean_values, subjects=df.loc[:, 'X1':'X9'])
mahalanobis.columns = list(range(1, 8))
df['mahalanobis'] = mahalanobis_classes(mahalanobis)
predict(df, 'mahalanobis')
print(f'Точность метода "Расстояние Махаланобиса" = {accuracy(df, "training", "mahalanobis").round(3)}')
df

Кластер 1: ['Алтайский край                          ', 'Кемеровская область — Кузбасс', 'Костромская область                     ', 'Курганская область                      ', 'Курская область                         ', 'Новгородская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Северная Осетия — Алания', 'Ростовская область                      ', 'Рязанская область                       ', 'Самарская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Иркутская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis
Наименование,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5


In [47]:
posterior = posterior_probabilities(centroids=mean_values, subjects=df.loc[:, 'X1':'X9'])
posterior.columns = list(range(1, 8))
df['posterior'] = posterior_classes(posterior)
predict(df, 'posterior')
print(f'Точность метода "Апостериорные вероятности" = {accuracy(df, "training", "posterior").round(3)}')
df

Кластер 1: ['Алтайский край                          ', 'Кемеровская область — Кузбасс', 'Костромская область                     ', 'Курганская область                      ', 'Курская область                         ', 'Новгородская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Северная Осетия — Алания', 'Ростовская область                      ', 'Рязанская область                       ', 'Самарская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Иркутская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior
Наименование,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5


In [48]:
df["Dis_analys_stepwise"] = discrim(df, forw_stepwise, 'training', 'k-means')
predict(df, 'Dis_analys_stepwise')
print(f'Точность Дискриминантного анализа с включением = {accuracy(df, "training", "Dis_analys_stepwise").round(3)}')
df

Кластер 1: ['Волгоградская область', 'Кемеровская область — Кузбасс', 'Кировская область                       ', 'Костромская область                     ', 'Курская область                         ', 'Мурманская область                      ', 'Новгородская область                    ', 'Оренбургская область                    ', 'Орловская область                       ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Мордовия                     ', 'Рязанская область                       ', 'Свердловская область                    ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тверская область                        ', 'Тюменская область без автономных округов', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Алтайский край                          ', 'Брянская область                        ', 'Владимирская область                    ', 'Вологодска

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior,Dis_analys_stepwise
Наименование,,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1,2
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5,5


In [49]:
mahalanobis_stepwise = mahalanobis_distances(
    centroids=mean_values.loc[:, forw_stepwise], subjects=df.loc[:, forw_stepwise])
mahalanobis_stepwise.columns = list(range(1, 8))
df['mahalanobis_stepwise'] = mahalanobis_classes(mahalanobis)
predict(df, 'mahalanobis_stepwise')
print(
    f'Точность метода "Расстояние Махаланобиса" с учетом пошагового дискриминантного анализа с включением" = {accuracy(df, "training", "mahalanobis_stepwise").round(3)}')
df

Кластер 1: ['Алтайский край                          ', 'Кемеровская область — Кузбасс', 'Костромская область                     ', 'Курганская область                      ', 'Курская область                         ', 'Новгородская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Северная Осетия — Алания', 'Ростовская область                      ', 'Рязанская область                       ', 'Самарская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Иркутская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior,Dis_analys_stepwise,mahalanobis_stepwise
Наименование,,,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1,2,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5,5,5


In [50]:
posterior_stepwise = posterior_probabilities(centroids=mean_values.loc[:, forw_stepwise], subjects=df.loc[:, forw_stepwise])
posterior_stepwise.columns = list(range(1, 8))
df['posterior_stepwise'] = posterior_classes(posterior)
predict(df, 'posterior_stepwise')
print(f'Точность метода "Апостериорные вероятности" с учетом пошагового дискриминантного анализа с включением = {accuracy(df, "training", "posterior_stepwise").round(3)}')
df
df

Кластер 1: ['Алтайский край                          ', 'Кемеровская область — Кузбасс', 'Костромская область                     ', 'Курганская область                      ', 'Курская область                         ', 'Новгородская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Северная Осетия — Алания', 'Ростовская область                      ', 'Рязанская область                       ', 'Самарская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Иркутская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior,Dis_analys_stepwise,mahalanobis_stepwise,posterior_stepwise
Наименование,,,,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1,2,1,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6,3,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5,5,5,5


In [51]:
df["Dis_analys_backward"] = discrim(df, forw_backward, 'training', 'k-means')
predict(df, 'Dis_analys_backward')
print(f'Точность Дискриминантного анализа с исключением = {accuracy(df, "training", "Dis_analys_backward").round(3)}')
df

Кластер 1: ['Волгоградская область', 'Кемеровская область — Кузбасс', 'Кировская область                       ', 'Костромская область                     ', 'Курская область                         ', 'Мурманская область                      ', 'Новгородская область                    ', 'Оренбургская область                    ', 'Орловская область                       ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Мордовия                     ', 'Рязанская область                       ', 'Свердловская область                    ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тверская область                        ', 'Тюменская область без автономных округов', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Алтайский край                          ', 'Брянская область                        ', 'Владимирская область                    ', 'Вологодска

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior,Dis_analys_stepwise,mahalanobis_stepwise,posterior_stepwise,Dis_analys_backward
Наименование,,,,,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1,2,1,1,2
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5,5,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5,5,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3,3,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6,3,6,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7,7,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2,2,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5,5,5,5,5


In [52]:
mahalanobis_backward = mahalanobis_distances(
    centroids=mean_values.loc[:, forw_backward], subjects=df.loc[:, forw_backward])
mahalanobis_stepwise.columns = list(range(1, 8))
df['mahalanobis_backward'] = mahalanobis_classes(mahalanobis)
predict(df, 'mahalanobis_backward')
print(
    f'Точность метода "Расстояние Махаланобиса" с учетом пошагового дискриминантного анализа с исключением" = {accuracy(df, "training", "mahalanobis_backward").round(3)}')
df

Кластер 1: ['Алтайский край                          ', 'Кемеровская область — Кузбасс', 'Костромская область                     ', 'Курганская область                      ', 'Курская область                         ', 'Новгородская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Северная Осетия — Алания', 'Ростовская область                      ', 'Рязанская область                       ', 'Самарская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Иркутская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior,Dis_analys_stepwise,mahalanobis_stepwise,posterior_stepwise,Dis_analys_backward,mahalanobis_backward
Наименование,,,,,,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1,2,1,1,2,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5,5,5,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5,5,5,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3,3,3,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6,3,6,6,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7,7,7,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2,2,2,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5,5,5,5,5,5


In [53]:
posterior_backward = posterior_probabilities(centroids=mean_values.loc[:, forw_backward], subjects=df.loc[:, forw_backward])
posterior_stepwise.columns = list(range(1, 8))
df['posterior_backward'] = posterior_classes(posterior)
predict(df, 'posterior_backward')
print(f'Точность метода "Апостериорные вероятности" с учетом пошагового дискриминантного анализа с исключением = {accuracy(df, "training", "posterior_backward").round(3)}')
df

Кластер 1: ['Алтайский край                          ', 'Кемеровская область — Кузбасс', 'Костромская область                     ', 'Курганская область                      ', 'Курская область                         ', 'Новгородская область                    ', 'Пермский край                           ', 'Республика Башкортостан', 'Республика Северная Осетия — Алания', 'Ростовская область                      ', 'Рязанская область                       ', 'Самарская область                       ', 'Смоленская область                      ', 'Тамбовская область                      ', 'Тюменская область без автономных округов', 'Удмуртская Республика', 'Челябинская область                     ', 'Ярославская область                     ']
Кластер 2: ['Брянская область                        ', 'Владимирская область                    ', 'Волгоградская область', 'Вологодская область                     ', 'Ивановская область                      ', 'Иркутская область                 

,k-means,X1,X2,X3,X4,X5,X6,X7,X8,X9,training,Dis_analys,mahalanobis,posterior,Dis_analys_stepwise,mahalanobis_stepwise,posterior_stepwise,Dis_analys_backward,mahalanobis_backward,posterior_backward
Наименование,,,,,,,,,,,,,,,,,,,,
Алтайский край,1,-0.745159,-0.507014,-0.229354,1.009379,1.584799,0.163745,0.064704,0.702768,-0.452985,NaN,1,1,1,2,1,1,2,1,1
Амурская область,5,-0.875390,0.685486,0.122922,1.054686,1.478816,1.872712,-0.403351,0.250717,1.269877,5.0,5,5,5,5,5,5,5,5,5
Архангельская область без автономного округа,1,-0.715752,1.154666,-0.140898,0.557898,-0.171211,0.137976,-0.537081,-0.265912,1.213204,NaN,5,5,5,5,5,5,5,5,5
Астраханская область,3,-0.266245,1.086244,-0.175726,-0.428830,-0.013651,-0.603997,0.551864,0.358348,-1.167066,NaN,3,3,3,3,3,3,3,3,3
Белгородская область,6,0.834418,-0.663407,-0.191904,-0.848439,-0.784912,0.220374,0.714250,-1.299171,-0.078942,NaN,3,6,6,3,6,6,3,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,7,2.094718,-1.552894,-0.490701,-1.689238,-2.063890,-2.709302,-0.049921,1.563817,-1.960489,7.0,7,7,7,7,7,7,7,7,7
Чувашская Республика — Чувашия,2,-0.009984,0.382474,-0.419584,-0.279214,-0.728112,-1.152608,0.599624,0.573610,-0.691012,NaN,2,2,2,2,2,2,2,2,2
Чукотский автономный округ,5,-2.110484,2.992288,0.681964,2.090231,-1.693058,1.155506,-2.065423,-1.191540,3.083416,NaN,5,5,5,5,5,5,5,5,5


In [54]:
def function(df, method):

    mean_values = df.groupby(method).mean().add_suffix('_mean')
    mean_values
    df_new = df.merge(mean_values, left_on=method, right_index=True, )
    df_new.loc[:, 'X1':'X9'] = (df_new.loc[:, 'X1':'X9'].values - df_new.loc[:, 'X1_mean':'X9_mean'].values) ** 2
    df_new['L_2'] = (df_new.loc[:, 'X1':'X9'].sum(axis=1))
    df_new.groupby(method).sum('L_2')
    table_function = pd.DataFrame(df_new.groupby(method).sum('L_2')['L_2'])
    sum_function = df_new.groupby(method).sum('L_2')['L_2'].sum()
    
    print(f'Функционал качсетва {method}: {sum_function}')

In [55]:
print(
    f'Точность Дискриминантного анализа = {accuracy(df, "training", "Dis_analys").round(3)}')
print(
    f'Точность Дискриминантного анализа с включением = {accuracy(df, "training", "Dis_analys_stepwise").round(3)}')
print(
    f'Точность Дискриминантного анализа с исключением = {accuracy(df, "training", "Dis_analys_backward").round(3)}')
print(
    f'Точность метода "Расстояние Махаланобиса" = {accuracy(df, "training", "mahalanobis").round(3)}')
print(
    f'Точность метода "Расстояние Махаланобиса" с учетом пошагового дискриминантного анализа с включением" = {accuracy(df, "training", "mahalanobis_stepwise").round(3)}')
print(
    f'Точность метода "Расстояние Махаланобиса" с учетом пошагового дискриминантного анализа с исключением" = {accuracy(df, "training", "mahalanobis_backward").round(3)}')
print(
    f'Точность метода "Апостериорные вероятности" = {accuracy(df, "training", "posterior").round(3)}')
print(
    f'Точность метода "Апостериорные вероятности" с учетом пошагового дискриминантного анализа с включением = {accuracy(df, "training", "posterior_stepwise").round(3)}')
print(
    f'Точность метода "Апостериорные вероятности" с учетом пошагового дискриминантного анализа с исключением = {accuracy(df, "training", "posterior_backward").round(3)}')

Точность Дискриминантного анализа = 0.929
Точность Дискриминантного анализа с включением = 0.929
Точность Дискриминантного анализа с исключением = 0.929
Точность метода "Расстояние Махаланобиса" = 0.929
Точность метода "Расстояние Махаланобиса" с учетом пошагового дискриминантного анализа с включением" = 0.929
Точность метода "Расстояние Махаланобиса" с учетом пошагового дискриминантного анализа с исключением" = 0.929
Точность метода "Апостериорные вероятности" = 0.929
Точность метода "Апостериорные вероятности" с учетом пошагового дискриминантного анализа с включением = 0.929
Точность метода "Апостериорные вероятности" с учетом пошагового дискриминантного анализа с исключением = 0.929


In [56]:
function(df, 'Dis_analys')
function(df, 'Dis_analys_stepwise')
function(df, 'Dis_analys_backward')
function(df, 'mahalanobis')
function(df, 'mahalanobis_stepwise')
function(df, 'mahalanobis_backward')
function(df, 'posterior')
function(df, 'posterior_stepwise')
function(df, 'posterior_backward')


Функционал качсетва Dis_analys: 471.50359196306795
Функционал качсетва Dis_analys_stepwise: 427.64700778981614
Функционал качсетва Dis_analys_backward: 427.64700778981614
Функционал качсетва mahalanobis: 428.7777328230197
Функционал качсетва mahalanobis_stepwise: 428.7777328230197
Функционал качсетва mahalanobis_backward: 428.7777328230197
Функционал качсетва posterior: 428.7777328230197
Функционал качсетва posterior_stepwise: 428.7777328230197
Функционал качсетва posterior_backward: 428.7777328230197
